In [1]:
import time
import numpy as np
import matplotlib.pyplot as plt
import seaborn
import torch
import pyreadr
import requests
import urllib.request
import math
import pandas as pd 
import random
import numpy as np
from sklearn.datasets import load_iris 
from scipy.ndimage import gaussian_filter
from sklearn import preprocessing
from sklearn import utils

    
import statistics
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import MinMaxScaler
from sklearn import datasets
from sklearn import svm
from torchvision import datasets, transforms
from sklearn import neighbors
from sklearn.metrics import confusion_matrix
from sklearn.decomposition import PCA
from sklearn.neighbors import KNeighborsClassifier

In [2]:
# We intend to predict the AC Power
# It should be done using Ambient temperature, module temperature, irradiation and time of day, because we these are forecasted
# The ML algorithm should only be trained on the above.

In [3]:
#Plant 1 datasets
P1G=pd.read_csv('Plant_1_Generation_Data.csv',na_values= '?')
P1W=pd.read_csv('Plant_1_Weather_Sensor_Data.csv',na_values= '?')

#Plant 2 datasets
P2G=pd.read_csv('Plant_2_Generation_Data.csv',na_values= '?')
P2W=pd.read_csv('Plant_2_Weather_Sensor_Data.csv',na_values= '?')

#Check for missing values, commented out because of no missing values. Imputation not necessary
#for col in range(P1W.shape[1]):
    # YOUR CODE HERE
#    n_miss = P1W.iloc[:, col].isna().sum()
#    print('Column {} has {} missing values.'.format(col, n_miss))
#P2G.isna().sum()
#P2W.dtypes
display(P2W)
display(P2G)

,DATE_TIME,PLANT_ID,SOURCE_KEY,AMBIENT_TEMPERATURE,MODULE_TEMPERATURE,IRRADIATION
0,2020-05-15 00:00:00,4136001,iq8k7ZNt4Mwm3w0,27.004764,25.060789,0.0
1,2020-05-15 00:15:00,4136001,iq8k7ZNt4Mwm3w0,26.880811,24.421869,0.0
2,2020-05-15 00:30:00,4136001,iq8k7ZNt4Mwm3w0,26.682055,24.427290,0.0
3,2020-05-15 00:45:00,4136001,iq8k7ZNt4Mwm3w0,26.500589,24.420678,0.0
4,2020-05-15 01:00:00,4136001,iq8k7ZNt4Mwm3w0,26.596148,25.088210,0.0
...,...,...,...,...,...,...
3254,2020-06-17 22:45:00,4136001,iq8k7ZNt4Mwm3w0,23.511703,22.856201,0.0
3255,2020-06-17 23:00:00,4136001,iq8k7ZNt4Mwm3w0,23.482282,22.744190,0.0
3256,2020-06-17 23:15:00,4136001,iq8k7ZNt4Mwm3w0,23.354743,22.492245,0.0
3257,2020-06-17 23:30:00,4136001,iq8k7ZNt4Mwm3w0,23.291048,22.373909,0.0


,DATE_TIME,PLANT_ID,SOURCE_KEY,DC_POWER,AC_POWER,DAILY_YIELD,TOTAL_YIELD
0,2020-05-15 00:00:00,4136001,4UPUqMRk7TRMgml,0.0,0.0,9425.000000,2.429011e+06
1,2020-05-15 00:00:00,4136001,81aHJ1q11NBPMrL,0.0,0.0,0.000000,1.215279e+09
2,2020-05-15 00:00:00,4136001,9kRcWv60rDACzjR,0.0,0.0,3075.333333,2.247720e+09
3,2020-05-15 00:00:00,4136001,Et9kgGMDl729KT4,0.0,0.0,269.933333,1.704250e+06
4,2020-05-15 00:00:00,4136001,IQ2d7wF4YD8zU1Q,0.0,0.0,3177.000000,1.994153e+07
...,...,...,...,...,...,...,...
67693,2020-06-17 23:45:00,4136001,q49J1IKaHRwDQnt,0.0,0.0,4157.000000,5.207580e+05
67694,2020-06-17 23:45:00,4136001,rrq4fwE8jgrTyWY,0.0,0.0,3931.000000,1.211314e+08
67695,2020-06-17 23:45:00,4136001,vOuJvMaM2sgwLmb,0.0,0.0,4322.000000,2.427691e+06
67696,2020-06-17 23:45:00,4136001,xMbIugepa2P7lBB,0.0,0.0,4218.000000,1.068964e+08


In [4]:
#Copying generation and weather data from plant 1
generation_data=P1G.copy()
weather_data=P1W.copy()

#Converting the DATE_TIME to a pandas datetime object
generation_data['DATE_TIME'] = pd.to_datetime(generation_data["DATE_TIME"])
weather_data['DATE_TIME'] = pd.to_datetime(weather_data["DATE_TIME"])

#Merging the 2 datasets and adding a column that contains the time of day in one column, which is more convenient for training.
df = pd.merge(generation_data.drop(columns=['PLANT_ID']), weather_data.drop(columns=['PLANT_ID', 'SOURCE_KEY']), on='DATE_TIME')
df['HOUR'] = pd.to_datetime(df['DATE_TIME'].astype(str)).dt.hour+(pd.to_datetime(df['DATE_TIME'].astype(str)).dt.minute)/60

#Dropping parameters, not usable for predicting AC_POWER
df=df.drop(['DATE_TIME','SOURCE_KEY','DC_POWER','DAILY_YIELD','TOTAL_YIELD'],axis=1)

#Show dataframe
df

,AC_POWER,AMBIENT_TEMPERATURE,MODULE_TEMPERATURE,IRRADIATION,HOUR
0,0.0,25.184316,22.857507,0.0,0.00
1,0.0,25.184316,22.857507,0.0,0.00
2,0.0,25.184316,22.857507,0.0,0.00
3,0.0,25.184316,22.857507,0.0,0.00
4,0.0,25.184316,22.857507,0.0,0.00
...,...,...,...,...,...
45675,0.0,21.909288,20.427972,0.0,23.75
45676,0.0,21.909288,20.427972,0.0,23.75
45677,0.0,21.909288,20.427972,0.0,23.75
45678,0.0,21.909288,20.427972,0.0,23.75


In [5]:
#max(df['AC_POWER'])

#Min max scale the data
#Making scaler
scaler=MinMaxScaler()
#Fitting scaler to the data
scaler.fit(df)
#Transform the data to values between 0 and 1
df_min_max=pd.DataFrame(scaler.transform(df))
#The transform removes the column labels, we therefore apply them again:
df_min_max.rename(columns={0: "AC_Power", 1: "Ambient_temperature", 2: "Module_temperature",3:"Irradiation",4:"Hour"},inplace=True)
display(df_min_max)

,AC_Power,Ambient_temperature,Module_temperature,Irradiation,Hour
0,0.0,0.322190,0.099506,0.0,0.0
1,0.0,0.322190,0.099506,0.0,0.0
2,0.0,0.322190,0.099506,0.0,0.0
3,0.0,0.322190,0.099506,0.0,0.0
4,0.0,0.322190,0.099506,0.0,0.0
...,...,...,...,...,...
45675,0.0,0.101709,0.048255,0.0,1.0
45676,0.0,0.101709,0.048255,0.0,1.0
45677,0.0,0.101709,0.048255,0.0,1.0
45678,0.0,0.101709,0.048255,0.0,1.0


In [6]:
y=df_min_max['AC_Power']
x=df_min_max.drop(['AC_Power'],axis=1)
lab = preprocessing.LabelEncoder()
y_transformed = lab.fit_transform(y)
print(np.shape(y))
print(np.shape(y_transformed))
X_train, X_test, y_train, y_test= train_test_split(x, y_transformed, test_size=0.25, random_state=42)

(45680,)
(45680,)


In [7]:
lr_model = LinearRegression()
lr_model.fit(X_train, y_train)


lr_score = lr_model.score(X_test, y_test)
print(f"Score is {lr_score*100:.4f} %")

Score is 97.9674 %


In [8]:
# import warnings filter
from warnings import simplefilter
# ignore all future warnings
simplefilter(action='ignore', category=FutureWarning)


from sklearn import neighbors

score_list = []
n_neighbors_list = []

# for loop to find best fitting n_neighbor value
for i in range(1, 21):
    knn = neighbors.KNeighborsRegressor(n_neighbors=i)
    knn.fit(X_train, y_train)

    knn_score = knn.score(X_test, y_test)

    score_list.append(knn_score)
    n_neighbors_list.append(i)

best_n_neighbor = n_neighbors_list[score_list.index(max(score_list))]
print(f"Best n_neighbor at {best_n_neighbor} with {max(score_list)*100:.4f} %")


Best n_neighbor at 16 with 99.0145 %


In [9]:
from sklearn.tree import DecisionTreeRegressor

dtr = DecisionTreeRegressor()
dtr.fit(X_train,y_train)

dtr_score = dtr.score(X_test, y_test)
print(f"Score is {dtr_score*100:.4f} %")

Score is 99.0281 %


In [10]:
from sklearn.ensemble import RandomForestRegressor

rfr = RandomForestRegressor()
rfr.fit(X_train,y_train)

rfr_score = rfr.score(X_test, y_test)
print(f"Score is {rfr_score*100:.4f} %")

Score is 99.0298 %
